# Process visualization data

This notebook processes the CovidCareMap US Healthcare System Capacity data into the format used with the `viz/heathcare-system-capacity` visualization.

In [32]:
import shutil
import os

from covidcaremap.data import published_data_dir, data_dir, processed_data_dir

In [39]:
viz_data_dir = os.path.join(data_dir(), '../viz/us-healthcare-system-capacity/data')

viz_input_dir = os.path.join(viz_data_dir, 'input')
viz_config_dir = os.path.join(viz_data_dir, 'config')

!mkdir -p $viz_input_dir
!mkdir -p $viz_config_dir

data_paths = [
    (os.path.join(published_data_dir(), x), os.path.join(viz_input_dir, x))
    for x in [
        'us_healthcare_capacity-county-CovidCareMap.geojson',
        'us_healthcare_capacity-facility-CovidCareMap.geojson',
        'us_healthcare_capacity-hrr-CovidCareMap.geojson',
        'us_healthcare_capacity-state-CovidCareMap.geojson'
    ]
] + [
    (os.path.join(processed_data_dir(), x), os.path.join(viz_config_dir, x))
    for x in [
        'ccm_county_min_max_values.json',
        'ccm_state_min_max_values.json',
        'ccm_hrr_min_max_values.json',
        'ccm_facility_min_max_values.json'
    ]
]

In [40]:
for src, dst in data_paths:
    print('Copying {} to {}'.format(src, dst))
    shutil.copy(src, dst)

Copying /opt/src/data/published/us_healthcare_capacity-county-CovidCareMap.geojson to /opt/src/data/../viz/us-healthcare-system-capacity/data/input/us_healthcare_capacity-county-CovidCareMap.geojson
Copying /opt/src/data/published/us_healthcare_capacity-facility-CovidCareMap.geojson to /opt/src/data/../viz/us-healthcare-system-capacity/data/input/us_healthcare_capacity-facility-CovidCareMap.geojson
Copying /opt/src/data/published/us_healthcare_capacity-hrr-CovidCareMap.geojson to /opt/src/data/../viz/us-healthcare-system-capacity/data/input/us_healthcare_capacity-hrr-CovidCareMap.geojson
Copying /opt/src/data/published/us_healthcare_capacity-state-CovidCareMap.geojson to /opt/src/data/../viz/us-healthcare-system-capacity/data/input/us_healthcare_capacity-state-CovidCareMap.geojson
Copying /opt/src/data/processed/ccm_county_min_max_values.json to /opt/src/data/../viz/us-healthcare-system-capacity/data/config/ccm_county_min_max_values.json
Copying /opt/src/data/processed/ccm_state_min_ma

In [30]:
!cd $viz_data_dir && ./process.sh

INFO:mbutil.util:1418 / 1675 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:1419 / 1675 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:1420 / 1675 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:1421 / 1675 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:1422 / 1675 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:1423 / 1675 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:1424 / 1675 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:1425 / 1675 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:1426 / 1675 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:1427 / 1675 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:1428 / 1675 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:1429 / 1675 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:1430 / 1675 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:1431 / 1675 tiles exported
DEBUG:mbutil.util:flipping
INFO:m

DEBUG:mbutil.util:flipping
INFO:mbutil.util:1544 / 1675 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:1545 / 1675 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:1546 / 1675 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:1547 / 1675 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:1548 / 1675 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:1549 / 1675 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:1550 / 1675 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:1551 / 1675 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:1552 / 1675 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:1553 / 1675 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:1554 / 1675 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:1555 / 1675 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:1556 / 1675 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:1557 / 1675 tiles exported
DEBUG:

INFO:mbutil.util:1672 / 1675 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:1673 / 1675 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:1674 / 1675 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:1675 / 1675 tiles exported
